# 滑动窗口提取数据-Q3

In [6]:
SSlabel=['C','E','H']
#SSlabel=['L', 'B', 'E', 'G', 'I', 'H', 'S', 'T']

In [7]:
# 滑动窗口提取氨基酸,slidsize必须是奇数

def GetSliding(filelist,slidsize):
    slidwins=[]
    
    #滑动取slidsize个
    indtmp=0
    while (indtmp+slidsize)<=len(filelist):
        if filelist[indtmp:indtmp+slidsize].count('emptyamni')==6 and filelist[indtmp:indtmp+slidsize][-1]=='emptyamni':
            slidwins.append(filelist[indtmp:indtmp+slidsize])
            indtmp=indtmp+7
        else:
            slidwins.append(filelist[indtmp:indtmp+slidsize])
            indtmp=indtmp+1
        
    return slidwins

In [8]:
#对氨基酸列表进行排序
def SortedFileList(filename,filelist):
    newfilelist=[]
    for ind1 in range(len(filelist)):
        if filename+'_cha'+str(ind1)+'_amni0' not in filelist:
            break
        for ind2 in range(len(filelist)):
            filenametmp=filename+'_cha'+str(ind1)+'_amni'+str(ind2)
            if filenametmp in filelist:
                newfilelist.append(filenametmp)
            else:
                break
    return newfilelist

In [9]:
def BuildSlidFea(filename,slidsize):

#     #filename='cb513'
#     filelist=open('0.Featrue/aminslist_%s_Q8v2'%filename).readlines()
#     print(filelist[:10])
    
#     #取氨基酸的名称
#     filelist=list(map(lambda file: file.split('&')[0] ,filelist))
    
#     print('sorte..........')
#     #对氨基酸列表进行排序
#     newfilelist=SortedFileList(filename,filelist)
#     #print(newfilelist[:10])
    
    addsize=int((slidsize-1)/2)
    newfilelist=[]
    
    chainindnow=0
    chainindpre=0
    for line in open('1.change/aminslist_%s_Q3.sort'%filename):
        line=line.replace('\n','')
        filenametmp=line.split('&')[0]
        chainindnow=int(((line.split('&')[0]).split('_')[1]).replace('cha',''))
        
        if chainindpre+1==chainindnow:
            #每条链后加（slidsize-1）/2个
            for i in range(addsize):
                newfilelist.append('emptyamni')
        newfilelist.append(filenametmp)
        chainindpre=int(((line.split('&')[0]).split('_')[1]).replace('cha',''))
    
    #前加（slidsize-1）/2个
    addsize=int((slidsize-1)/2)
    for i in range(addsize):
        newfilelist.insert(0, 'emptyamni') 

    #后加（slidsize-1）/2个
    for i in range(addsize):
        newfilelist.append('emptyamni') 
    
    #-----------------------
    #窗口滑动
    slidwins=GetSliding(newfilelist,slidsize)

    print(slidwins[:3])
    print(slidwins[-3:])

    #----------------------------
    #读取氨基酸列表的所有特征
    print(filename)
    allaminfea=open('1.change/aminsinfo_%s_Q3.sort'%filename).readlines()
    
    #按字典存储氨基酸特征
    allaminfeadic={}
    def buildaminfeadic(line):
        line=line.replace('\n','')
        line=line.split('&')
        return line[0],line[2:]

    for aminname,aminfea in  list(map(buildaminfeadic,allaminfea)):
        allaminfeadic[aminname]=aminfea

    # #添加空氨基酸特征
    allaminfeadic['emptyamni']=['None', '0|0|0|0.0|0.0|0.0|0.0|0.0', '0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0']
    # print(allaminfeadic['cb513_cha0_amni0'])

    #------------------------------
    #提取滑动窗口的特征和标签，并存储
    fw_fea=open('2.Sliding/%s_slidfea_Q3'%filename,'w')
    fw_lab=open('2.Sliding/%s_slidlab_Q3'%filename,'w')
    fw_wind=open('2.Sliding/%s_slidwind_Q3'%filename,'w')

    for slidwin in slidwins:
        slidwinfea=[]

        #读取一个窗口的所有特征
        for amino in slidwin:
            #读取某个氨基酸的特征
            aminofea=allaminfeadic[amino][1].split('|')+allaminfeadic[amino][2].split('|')
            slidwinfea.extend(aminofea)
        
        #print('here',slidwin[int((len(slidwin)-1)/2)],slidwin)
        aminolab=SSlabel.index(allaminfeadic[slidwin[int((len(slidwin)-1)/2)]][0])

        fw_fea.write(','.join(list(map(str,slidwinfea)))+'\n')
        fw_lab.write(str(aminolab)+'\n')
        fw_wind.write('-'.join(slidwin)+'\n')
    fw_fea.close()
    fw_lab.close()
    fw_wind.close()

In [10]:
slidsize=13
BuildSlidFea('cb513',slidsize)
BuildSlidFea('cullpdb',slidsize)

[['emptyamni', 'emptyamni', 'emptyamni', 'emptyamni', 'emptyamni', 'emptyamni', 'cb513_cha0_amni0', 'cb513_cha0_amni1', 'cb513_cha0_amni2', 'cb513_cha0_amni3', 'cb513_cha0_amni4', 'cb513_cha0_amni5', 'cb513_cha0_amni6'], ['emptyamni', 'emptyamni', 'emptyamni', 'emptyamni', 'emptyamni', 'cb513_cha0_amni0', 'cb513_cha0_amni1', 'cb513_cha0_amni2', 'cb513_cha0_amni3', 'cb513_cha0_amni4', 'cb513_cha0_amni5', 'cb513_cha0_amni6', 'cb513_cha0_amni7'], ['emptyamni', 'emptyamni', 'emptyamni', 'emptyamni', 'cb513_cha0_amni0', 'cb513_cha0_amni1', 'cb513_cha0_amni2', 'cb513_cha0_amni3', 'cb513_cha0_amni4', 'cb513_cha0_amni5', 'cb513_cha0_amni6', 'cb513_cha0_amni7', 'cb513_cha0_amni8']]
[['cb513_cha513_amni45', 'cb513_cha513_amni46', 'cb513_cha513_amni47', 'cb513_cha513_amni48', 'cb513_cha513_amni49', 'cb513_cha513_amni50', 'cb513_cha513_amni51', 'cb513_cha513_amni52', 'cb513_cha513_amni53', 'emptyamni', 'emptyamni', 'emptyamni', 'emptyamni'], ['cb513_cha513_amni46', 'cb513_cha513_amni47', 'cb513_ch

slidsize=13
filename='cb513'

addsize=int((slidsize-1)/2)

newfilelist=[]
chainindnow=0
chainindpre=0
for line in open('0.Featrue/aminslist_%s_Q8v3'%filename):
    line=line.replace('\n','')
    filenametmp=line.split('&')[0]
    chainindnow=int(((line.split('&')[0]).split('_')[1]).replace('cha',''))
        
    if chainindpre+1==chainindnow:
        #每条链后加（slidsize-1）/2个
        for i in range(addsize):
            newfilelist.append('emptyamni')
    newfilelist.append(filenametmp)
    chainindpre=int(((line.split('&')[0]).split('_')[1]).replace('cha',''))
print(newfilelist[:100])

#前加（slidsize-1）/2个
addsize=int((slidsize-1)/2)
for i in range(addsize):
    newfilelist.insert(0, 'emptyamni') 

#后加（slidsize-1）/2个
for i in range(addsize):
    newfilelist.append('emptyamni') 
#窗口滑动
slidwins=GetSliding(newfilelist,slidsize)

print(slidwins[65:70])